In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
from openslide import OpenSlide

In [3]:
from probreg import cpd
from probreg import callbacks
from probreg import transformation as tf

In [4]:
plt.rcParams.update({'font.size': 25})

patch_size = 1024

In [5]:
def add_help_fields(frame):
    
    frame["image_name_stem"] = [Path(image_name).stem for image_name in frame["image_name"]]
    
    frame["patient_id"] = [name.split("_")[2] for name in frame["image_name"]]

    frame["x1"] = [json.loads(vector.replace("\'","\""))['x1'] for vector in frame["vector"]]
    frame["y1"] = [json.loads(vector.replace("\'","\""))['y1'] for vector in frame["vector"]]

    frame["x2"] = [json.loads(vector.replace("\'","\""))['x2'] for vector in frame["vector"]]
    frame["y2"] = [json.loads(vector.replace("\'","\""))['y2'] for vector in frame["vector"]]

    frame["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    frame["center"] = [[center_x, center_y] for center_x, center_y in zip(frame["center_x"], frame["center_y"])]

    frame["anno_width"] = [x2-x1 for x1, x2 in zip(frame["x1"], frame["x2"])]
    frame["anno_height"]= [y2-y1 for y1, y2 in zip(frame["y1"], frame["y2"])]
    
    return frame

In [6]:
gt_annotations = add_help_fields(pd.read_csv("GT.csv"))
gt_annotations.head()

,scanner,image_id,image_name,image_type,image_width,image_height,id,vector,unique_identifier,annotation_type,...,patient_id,x1,y1,x2,y2,center_x,center_y,center,anno_width,anno_height
0,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755406,"{'x1': 20354, 'x2': 20408, 'y1': 16752, 'y2': ...",95183f6e-0a00-4de4-9c97-fedfec655a27,384,...,380609B,20354,16752,20408,16806,20381.0,16779.0,"[20381.0, 16779.0]",54,54
1,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755407,"{'x1': 20256, 'x2': 20310, 'y1': 35448, 'y2': ...",cc3def2b-5631-4619-ab08-51f2a69c667d,385,...,380609B,20256,35448,20310,35502,20283.0,35475.0,"[20283.0, 35475.0]",54,54
2,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755408,"{'x1': 20445, 'x2': 20499, 'y1': 54139, 'y2': ...",2d3de917-e55a-470f-8af2-10eb49b69687,386,...,380609B,20445,54139,20499,54193,20472.0,54166.0,"[20472.0, 54166.0]",54,54
3,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755409,"{'x1': 26495, 'x2': 26549, 'y1': 71854, 'y2': ...",611f9d35-6cdb-42ef-bc38-2f44c9aaa257,387,...,380609B,26495,71854,26549,71908,26522.0,71881.0,"[26522.0, 71881.0]",54,54
4,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,2755410,"{'x1': 35443, 'x2': 35497, 'y1': 90091, 'y2': ...",bf3ef0d3-5153-4693-96ba-96047cb3ae24,388,...,380609B,35443,90091,35497,90145,35470.0,90118.0,"[35470.0, 90118.0]",54,54


In [13]:
results, quality = [], []


for patient_id in tqdm(gt_annotations["patient_id"].unique()):
    
    patient_annos = gt_annotations[gt_annotations["patient_id"] == patient_id]
    
    source_annos = patient_annos[patient_annos["scanner"] == "Aperio"]        
    
    intersections = list(set(source_annos["type_name"]).intersection(patient_annos["type_name"]))
    
    source_annos = source_annos[source_annos["type_name"].isin(intersections)]
    
    source_anno = source_annos.iloc[0]
    
    for scanner in ['2.0HT', 'Axio', 'S210']:
        
        target_annos = patient_annos[patient_annos["scanner"] == scanner]
        target_annos = target_annos[target_annos["type_name"].isin(intersections)]
        
        target_anno = target_annos.iloc[0]
        
        source = np.array([a for a in source_annos["center"]])
        target = np.array([a for a in target_annos["center"]])

        tf_param, sigma2, q = cpd.registration_cpd(source, target, 'affine')
        mean_reg_error = np.linalg.norm(tf_param.transform(source)-target, axis=1).mean()
        
        mpp_x_scale = tf_param.b[0][0]
        mpp_y_scale = tf_param.b[1][1]
        
        
        target_scanner = target_anno.scanner 
        image_id = target_anno.image_id 
        image_name = target_anno.image_name 
        image_width, image_height = target_anno.image_width, target_anno.image_height
        
        image_type = "CCMCT" if "CCMCT" in image_name else "Cyto"
        
        quality.append([patient_id, source_anno.image_name, target_anno.image_name, 
                        int(q), int(sigma2), image_type, tf_param.b, tf_param.t, mean_reg_error])
        
        for id, source_anno in source_annos.iterrows():          
            
            # trans center
            box_1 = [source_anno.center_x, source_anno.center_y]
            trans_box_1 = tf_param.transform([box_1])[0]
            
            # trans width and hight
            new_width, new_height = source_anno.anno_width * mpp_x_scale, source_anno.anno_height * mpp_y_scale
            
            trans_box = [trans_box_1[0], trans_box_1[1], new_width, new_height]
            
            new_x1 = int(trans_box[0] - trans_box[2] // 2)
            new_y1 = int(trans_box[1] - trans_box[3] // 2)
            new_x2 = int(trans_box[0] + trans_box[2] // 2)
            new_y2 = int(trans_box[1] + trans_box[3] // 2)
            
            vector = {'x1': new_x1, 'x2': new_x2, 'y1': new_y1, 'y2': new_y2}
            
            row = [target_scanner, image_id, image_name, image_type, image_width, image_height,
                    vector, source_anno.unique_identifier, source_anno.annotation_type, 
                   source_anno.type_name, "GT_Registration"]
            
            results.append(row)

quality = pd.DataFrame(quality, columns=["patient_id", "source", "target","q", "sigma2", "image_type", "b", "t", "mean_reg_error"])

results = pd.DataFrame(results, columns=["scanner", "image_id", "image_name", "image_type", "image_width", 
                                         "image_height", "vector", "unique_identifier", "annotation_type", "type_name", "method"])
results.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 20.33it/s]


,scanner,image_id,image_name,image_type,image_width,image_height,vector,unique_identifier,annotation_type,type_name,method
0,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 20353, 'x2': 20407, 'y1': 16756, 'y2': ...",95183f6e-0a00-4de4-9c97-fedfec655a27,384,L0,GT_Registration
1,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 20253, 'x2': 20307, 'y1': 35451, 'y2': ...",cc3def2b-5631-4619-ab08-51f2a69c667d,385,L1,GT_Registration
2,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 20444, 'x2': 20498, 'y1': 54140, 'y2': ...",2d3de917-e55a-470f-8af2-10eb49b69687,386,L2,GT_Registration
3,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 26498, 'x2': 26552, 'y1': 71856, 'y2': ...",611f9d35-6cdb-42ef-bc38-2f44c9aaa257,387,L3,GT_Registration
4,2.0HT,10410,N2_CCMCT_380609B_1.ndpi,CCMCT,147456,107008,"{'x1': 35440, 'x2': 35494, 'y1': 90089, 'y2': ...",bf3ef0d3-5153-4693-96ba-96047cb3ae24,388,L4,GT_Registration


In [12]:
results.to_csv("GT_Registration.csv", index=False)

In [19]:
quality[["patient_id", "target", "mean_reg_error"]]

,patient_id,target,mean_reg_error
0,380609B,N2_CCMCT_380609B_1.ndpi,3.793803
1,380609B,Z_CCMCT_380609B_1.tif,4.556194
2,380609B,N1_CCMCT_380609B_1.ndpi,3.763742
3,518711B,N2_CCMCT_518711B_1.ndpi,3.245883
4,518711B,Z_CCMCT_518711B_1.tif,5.397241
5,518711B,N1_CCMCT_518711B_1.ndpi,3.393638
6,183715A,N2_CCMCT_183715A_1.ndpi,3.879318
7,183715A,Z_CCMCT_183715A_1.tif,4.184150
8,183715A,N1_CCMCT_183715A_1.ndpi,3.107848
9,29609B,N2_CCMCT_29609B_1.ndpi,3.372083


In [15]:
quality["mean_reg_error"].sum()

194.43461350602308